In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('C:/Users/HP/OneDrive/Chatbot/chatbot_model.h5')
import json
import random
intents = json.loads(open('C:/Users/HP/OneDrive/Chatbot/project.json').read())
words = pickle.load(open('C:/Users/HP/OneDrive/Chatbot/words.pkl','rb'))
classes = pickle.load(open('C:/Users/HP/OneDrive/Chatbot/classes.pkl','rb'))


def callback(event):
    webbrowser.open_new(event.widget.cget("text"))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res



#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        #res1='hi'
        #if res=='Directing you to placements module.':
            #ChatLog.insert(END, "Bot: "+"subitra")
        #else:
        ChatLog.insert(END, "GRIETBot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 




base = Tk()
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)


from tkinter import *
import webbrowser

def callback(url):
    webbrowser.open_new(url)

def popup_bonus():
    top = Toplevel()
    link1 = Label(top, text="GRIET Website", fg="blue", cursor="hand2")
    link1.pack()
    link1.bind("<Button-1>", lambda e: callback("http://www.griet.ac.in/"))
    
    link2 = Label(top, text="GRIET Placements", fg="blue", cursor="hand2")
    link2.pack()
    link2.bind("<Button-1>", lambda e: callback("http://www.griet.ac.in/placements.php"))
    
    link3 = Label(top, text="Courses and Intake", fg="blue", cursor="hand2")
    link3.pack()
    link3.bind("<Button-1>", lambda e: callback("http://www.griet.ac.in/programmes.php"))
    
    link4 = Label(top, text="Syllabus", fg="blue", cursor="hand2")
    link4.pack()
    link4.bind("<Button-1>", lambda e: callback("http://www.griet.ac.in/syllabus.php"))
    
    link5 = Label(top, text="EAMCET Rank", fg="blue", cursor="hand2")
    link5.pack()
    link5.bind("<Button-1>", lambda e: callback("http://www.griet.ac.in/eamcet_lastrank.php"))
    
    link6 = Label(top, text="Fee Structure", fg="blue", cursor="hand2")
    link6.pack()
    link6.bind("<Button-1>", lambda e: callback("http://www.griet.ac.in/fee_structure.php"))
    
    link7 = Label(top, text="Moodle", fg="blue", cursor="hand2")
    link7.pack()
    link7.bind("<Button-1>", lambda e: callback("http://www.moodle.griet.ac.in/"))
    
    link8 = Label(top, text="Exam Notification", fg="blue", cursor="hand2")
    link8.pack()
    link8.bind("<Button-1>", lambda e: callback("http://www.exambranch.griet.ac.in/"))
    
    link9 = Label(top, text="Results", fg="blue", cursor="hand2")
    link9.pack()
    link9.bind("<Button-1>", lambda e: callback("http://www.exambranch.griet.ac.in/"))

btn = Button(base, text = 'HELP', bd = '6',command = popup_bonus)        
btn.pack(side = 'top')


base.mainloop()


Using TensorFlow backend.
